In [ ]:
import numpy as np
import scipy as sp
import scipy.signal as sig
import altair as alt
import pandas as pd
from pathlib import Path  # pathlib is seriously awesome!
import sys
import sklearn.decomposition as sld

In [ ]:
DRS_file=pd.read_csv('All_RSC_Data.csv')
rs_file=pd.ExcelFile('2023-07-14_random_sample.xlsx')
#rs stands for random sample
rs_sheet_info=pd.read_excel(rs_file,'samples-354-U1449_U1450_U1451_U')
rs_sheet_lith=pd.read_excel(rs_file,'random_sample')
#rs_sheet = the specific sheet using to merge
MiddleDepth=(rs_sheet_info['Top depth CSF-A (m)']+rs_sheet_info['Bottom depth CSF-A (m)'])/2
rs_sheet_info.insert(11,'depth-m',MiddleDepth)
#calculating middle depth from top and bottom depth of core section and inserting values into sheet as new column
DRS_data=pd.merge(rs_sheet_info,DRS_file,on=['Site','Hole','depth-m'])
DRS_data=pd.merge(DRS_data,rs_sheet_lith,left_on=['ID'],right_on=['name'],how='left')
DRS_data.to_csv('DRS_data.csv',index=False)

In [ ]:
raw_data=pd.read_csv('DRS_data.csv') # <--Replace the folder and file names with the merge results csv file 

coercivity=pd.read_csv('2023-09-29_irm_gradient.csv')
coercivity.drop(coercivity[coercivity['max_d2']>2000].index, inplace=True)
coercivity['depth-m']=coercivity['depth-m']-1+0.01
coercivity['site']=coercivity.site+coercivity.hole
coercivity=coercivity.drop('hole', axis=1)

hysteresis=pd.read_csv('2023-09-29_hysteresis_parameters_by_lithology.csv')
hysteresis['depth-m']=hysteresis['top depth csf-a (m)']+0.01
hysteresis['site']=hysteresis.site+hysteresis.hole
hysteresis=hysteresis.drop('hole', axis=1)

In [ ]:
v=raw_data.columns.values

In [ ]:
raw_data['site_hole']=raw_data.Site+raw_data.Hole
print(raw_data.site_hole.unique())
site_holes=raw_data.site_hole.unique()
first_derivatives=dict(zip(raw_data.site_hole.unique(),np.zeros(raw_data.site_hole.unique().shape)))
km_first_derivatives=dict(zip(raw_data.site_hole.unique(),np.zeros(raw_data.site_hole.unique().shape)))

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise (ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise (ValueError, "Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise (ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

def first_derivative_smooth(reflectance,wavelength,wl=5,filter='flat'):
    first=int((wl-1)/2)
    smoothed_reflectance = smooth(reflectance,wl,filter)[first:-first]
    return pd.Series(data=np.gradient(smoothed_reflectance,wavelength),index=wavelength)

def first_derivative(reflectance,wavelength):
    return pd.Series(data=np.gradient(reflectance,wavelength),index=wavelength)

In [ ]:
smooth_param=99
def kubelka_munk(x):
    x = x/100
    return ((1-x)**2)/(2*x)

data_columns=['depth-m']+[str(x) for x in np.linspace(400,850,num=226,dtype='int')]

for sh in site_holes:
    print('Working on: ',sh)
    sh_data=raw_data.loc[(raw_data.site_hole==sh),[(x in data_columns) for x in raw_data.columns]]
    sh_kubelka_munk=sh_data
    sh_kubelka_munk.loc[:,'400':'850']=sh_data.loc[:,'400':'850'].applymap(kubelka_munk)
    wavelength=[float(i) for i in sh_data.drop('depth-m',axis=1).columns]
    sh_grad_smooth=sh_data.drop('depth-m',axis=1).apply(first_derivative_smooth,axis=1,raw=True,wavelength=wavelength,wl=smooth_param)
    sh_grad_smooth['depth-m']=sh_data['depth-m']
    first_derivatives[sh]=sh_grad_smooth
    
    sh_grad_km_smooth=sh_kubelka_munk.drop('depth-m',axis=1).apply(first_derivative_smooth,axis=1,raw=True,wavelength=wavelength,wl=smooth_param)
    sh_grad_km_smooth['depth-m']=sh_kubelka_munk['depth-m']
    km_first_derivatives[sh]=sh_grad_km_smooth
   
print('Finished.')

In [ ]:
km_first_derivatives_df=pd.DataFrame.from_dict({(x,y):km_first_derivatives[x][y]
                                                for x in km_first_derivatives.keys()
                                                for y in km_first_derivatives[x].keys()},
                                            orient='columns')
km_first_derivatives_df=km_first_derivatives_df.stack(level=0)
km_first_derivatives_df=km_first_derivatives_df.droplevel(0)
km_first_derivatives_df=km_first_derivatives_df.drop_duplicates()

site_info_DRS=km_first_derivatives_df['depth-m'].reset_index()
#takes index and depth into new data frame and makes them columns
site_info_DRS=site_info_DRS.rename(columns={'index':'site'})
site_info_DRS['label']=site_info_DRS['site']+'-'+site_info_DRS['depth-m'].astype('str')
#adds new column and assigns to data from site_info (astype- convert to string)
site_info_DRS.reset_index(inplace=True)

km_first_derivatives_df=km_first_derivatives_df.drop(('depth-m'),axis=1)
min_value_DRS=km_first_derivatives_df.values.min(axis=None)
km_first_derivatives_shifted=km_first_derivatives_df-min_value_DRS

km_first_derivatives_plus_site_info=km_first_derivatives_df.reset_index()
km_first_derivatives_plus_site_info=km_first_derivatives_plus_site_info.rename(columns={'index':'site'})
km_first_derivatives_plus_site_info=pd.merge(site_info_DRS,km_first_derivatives_plus_site_info,on=['site'])

km_derivatives_coercivity=pd.merge(km_first_derivatives_plus_site_info,coercivity, on=['site','depth-m'])
site_info_DRS_coercivity=km_derivatives_coercivity[['site','depth-m','label']].reset_index()


km_derivatives_coercivity_hysteresis=pd.merge(km_derivatives_coercivity,hysteresis, on=['site','depth-m'])
site_info_DRS_coercivity_hysteresis=km_derivatives_coercivity_hysteresis[['site','depth-m','label']].reset_index()

km_derivatives_coercivity=km_derivatives_coercivity.drop(['index','depth-m','site','label','max_d2','core','section','interval','Unnamed: 0','id'], axis=1)
min_value_DRS_coercivity=km_derivatives_coercivity.values.min(axis=None)
km_derivatives_coercivity_shifted=km_derivatives_coercivity-min_value_DRS_coercivity

km_derivatives_coercivity_hysteresis=km_derivatives_coercivity_hysteresis.drop(['index','depth-m','site','label','max_d2',
    'core_x','section_x','interval_x','Unnamed: 0_x','id_x','Unnamed: 0_y','id_y','core_y','section_y','interval_y','top [cm]','bottom [cm]','lithology principal name','top depth csf-a (m)'], axis=1)
min_value_DRS_coercivity_hysteresis=km_derivatives_coercivity_hysteresis.values.min(axis=None)
km_derivatives_coercivity_hysteresis_shifted=km_derivatives_coercivity_hysteresis-min_value_DRS_coercivity_hysteresis

In [ ]:
u1450a_first_derivative=first_derivatives['U1450A'].melt(id_vars=['depth-m'])
u1451b_first_derivative=first_derivatives['U1451B'].melt(id_vars=['depth-m'])
u1450a_km_first_derivative=km_first_derivatives['U1450A'].melt(id_vars=['depth-m'])
u1451b_km_first_derivative=km_first_derivatives['U1451B'].melt(id_vars=['depth-m'])

u1450a_first_derivative.columns=['depth','wavelength','value']
u1451b_first_derivative.columns=['depth','wavelength','value']
u1450a_km_first_derivative.columns=['depth','wavelength','value']
u1451b_km_first_derivative.columns=['depth','wavelength','value']

u1450a_first_derivative['wavelength']=pd.to_numeric(u1450a_first_derivative['wavelength'])
u1450a_first_derivative['depth']=pd.to_numeric(u1450a_first_derivative['depth'])
u1451b_first_derivative['wavelength']=pd.to_numeric(u1451b_first_derivative['wavelength'])
u1451b_first_derivative['depth']=pd.to_numeric(u1451b_first_derivative['depth'])
u1450a_km_first_derivative['wavelength']=pd.to_numeric(u1450a_km_first_derivative['wavelength'])
u1450a_km_first_derivative['depth']=pd.to_numeric(u1450a_km_first_derivative['depth'])
u1451b_km_first_derivative['wavelength']=pd.to_numeric(u1451b_km_first_derivative['wavelength'])
u1451b_km_first_derivative['depth']=pd.to_numeric(u1451b_km_first_derivative['depth'])

In [ ]:
print(first_derivatives['U1450A'].index)

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook

fig, ax = plt.subplots(len(site_holes), 1, figsize=(12,8), dpi=100)
fig.suptitle('First Derivatives of KM Function')

for i,sh in enumerate(site_holes):
    print('Working on: ',sh)
    km_first_derivatives[sh].T.rename(columns=km_first_derivatives[sh].T.loc['depth-m',:]).drop(index='depth-m').plot(color='k',alpha=0.1,ax=ax[i])
    ax[i].set_title(sh)
plt.show()

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
model=sld.NMF(n_components=5,init='nndsvda', solver='cd', max_iter=5000, random_state=0)
W_1 = model.fit_transform(km_derivatives_coercivity_hysteresis_shifted) # Model loadings, Data 1 (unrotated)
H_1 = model.components_ # Model factors, Data 1 (unrotated)

model_loadings_1=pd.DataFrame(W_1,columns=['A','B','C','D','E']).div(pd.DataFrame(W_1,columns=['A','B','C','D','E']).sum(axis=1),axis=0)
model_factors_1=pd.DataFrame(np.transpose(H_1),columns=['A','B','C','D','E'])

model_loadings_1_m=model_loadings_1.reset_index().melt(id_vars='index')
model_factors_1_m=model_factors_1.reset_index().melt(id_vars='index')


model_loadings_1_m=pd.merge(site_info_DRS_coercivity_hysteresis, model_loadings_1_m,on='index')
print(model_loadings_1_m)

model_loadings_1_chart=alt.Chart(model_loadings_1_m).mark_bar().encode(
    x='depth-m:Q',
    y='value:Q',
    facet='site:N',
    color='variable:N',
    tooltip='depth-m'
).properties(
    height=300,
    width=900,
    title='Model 1 (unrotated) - DRS + Coercivity + Hysteresis'
)
display(model_loadings_1_chart)

model_1_factors_chart=alt.Chart(model_factors_1_m).mark_line(strokeDash=[1,1]).encode(
    x=alt.X('index:Q',title='wavelength',scale=alt.Scale(domain=(0,850))),
    y='value:Q',
    color='variable:N'
).properties(
    width=600,
    height=300,
    title='Model 1 (unrotated) - DRS + Coercivity + Hysteresis'
)
display( (model_1_factors_chart))

model_1_error=km_derivatives_coercivity_hysteresis_shifted-model.inverse_transform(W_1)
model_1_error_m=pd.DataFrame(model_1_error).reset_index().melt(id_vars='index')
model_1_error_chart=alt.Chart(model_1_error_m).mark_line(opacity=0.5,color='black').encode(
    x='variable:Q',
    y='value:Q',
    detail='index:O'
).properties(
    title='Model 1 Error - DRS + Coercivity + Hysteresis',
    width=600,
    height=300
)
display(model_1_error_chart)

In [ ]:
model=sld.NMF(n_components=5, init='nndsvda', solver='cd', max_iter=5000, random_state=0)
W_2 = model.fit_transform(km_derivatives_coercivity_shifted) # Model loadings, Data 1 (unrotated)
H_2 = model.components_ # Model factors, Data 1 (unrotated)

model_loadings_2=pd.DataFrame(W_2,columns=['A','B','C','D','E']).div(pd.DataFrame(W_2,columns=['A','B','C','D','E']).sum(axis=1),axis=0)
model_factors_2=pd.DataFrame(np.transpose(H_2),columns=['A','B','C','D','E'])

model_loadings_2_m=model_loadings_2.reset_index().melt(id_vars='index')
model_factors_2_m=model_factors_2.reset_index().melt(id_vars='index')

model_loadings_2_m=pd.merge(site_info_DRS_coercivity, model_loadings_2_m,on='index')

model_loadings_2_chart=alt.Chart(model_loadings_2_m).mark_bar().encode(
    x='depth-m:Q',
    y='value:Q',
    facet='site:N',
    color='variable:N',
    tooltip='depth-m'
).properties(
    height=300,
    width=900,
    title='Model 2 (unrotated) - DRS + Coercivity'
)
display(model_loadings_2_chart)

model_2_factors_chart=alt.Chart(model_factors_2_m).mark_line(strokeDash=[1,1]).encode(
    x=alt.X('index:Q',title='wavelength',scale=alt.Scale(domain=(0,850))),
    y='value:Q',
    color='variable:N'
).properties(
    width=600,
    height=300,
    title='Model 2 (unrotated) - DRS + Coercivity'
)
display( (model_2_factors_chart))

model_2_error=km_derivatives_coercivity_shifted-model.inverse_transform(W_2)
model_2_error_m=pd.DataFrame(model_2_error).reset_index().melt(id_vars='index')
model_2_error_chart=alt.Chart(model_2_error_m).mark_line(opacity=0.5,color='black').encode(
    x='variable:Q',
    y='value:Q',
    detail='index:O'
).properties(
    title='Model 2 Error - DRS + Coercivity',
    width=600,
    height=300
)
display(model_2_error_chart)

error_squared=np.square(model_2_error)
error_squared_mean=error_squared.stack().mean()
print(error_squared_mean)

In [ ]:
model=sld.NMF(n_components=5,init='nndsvda', solver='cd', max_iter=5000, random_state=0)
W_3 = model.fit_transform(km_first_derivatives_shifted) # Model loadings, Data 1 (unrotated)
H_3 = model.components_ # Model factors, Data 1 (unrotated)

model_loadings_3=pd.DataFrame(W_3,columns=['A','B','C','D','E']).div(pd.DataFrame(W_3,columns=['A','B','C','D','E']).sum(axis=1),axis=0)
model_factors_3=pd.DataFrame(np.transpose(H_3),columns=['A','B','C','D','E'],index=wavelength)

model_loadings_3_m=model_loadings_3.reset_index().melt(id_vars='index')
model_factors_3_m=model_factors_3.reset_index().melt(id_vars='index')


model_loadings_3_m=pd.merge(site_info_DRS, model_loadings_3_m,on='index')

model_loadings_3_chart=alt.Chart(model_loadings_3_m).mark_bar().encode(
    x='depth-m:Q',
    y='value:Q',
    facet='site:N',
    color='variable:N',
    tooltip='depth-m'
).properties(
    height=300,
    width=900,
    title='Model 3 (unrotated) - DRS Only'
)
display(model_loadings_3_chart)

model_3_factors_chart=alt.Chart(model_factors_3_m).mark_line(strokeDash=[1,1]).encode(
    x=alt.X('index:Q',title='wavelength',scale=alt.Scale(domain=(400,850))),
    y='value:Q',
    color='variable:N'
).properties(
    width=600,
    height=300,
    title='Model 3 (unrotated) - DRS Only'
)
display( (model_3_factors_chart))

model_3_error=km_first_derivatives_shifted-model.inverse_transform(W_3)
model_3_error_m=pd.DataFrame(model_3_error).reset_index().melt(id_vars='index')

model_3_error_chart=alt.Chart(model_3_error_m).mark_line(opacity=0.5,color='black').encode(
    x='variable:Q',
    y='value:Q',
    detail='index:O'
).properties(
    title='Model 3 Error - DRS Only',
    width=600,
    height=300
)
display(model_3_error_chart)

error_squared=np.square(model_3_error)
error_squared_mean=error_squared.stack().mean()
print(error_squared_mean)

In [ ]:
model_loadings_2_m.to_csv('model loadings merge check',index=False)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

u1450a_chart=alt.Chart(u1450a_first_derivative).mark_line(opacity=0.1).encode(
    x=alt.X('wavelength:Q'),
    y=alt.Y('value:Q'),
    color='depth:Q'
)

display (u1450a_chart)

In [ ]:
def kubelka_munk(x):
    x = x/100
    return ((1-x)**2)/(2*x)

km_transformed = raw_data.applymap(kubelka_munk)
km_transformed['wavelength']=raw_data['wavelength']
print(km_transformed.head())

In [ ]:
km_200=alt.Chart(km_transformed.loc[:,['wavelength',100,1,0.1,0]].melt(id_vars=['wavelength'])).mark_line().encode(
    x=alt.X('wavelength'),
    y=alt.Y('value',axis=alt.Axis(title='K-M function')),
    color='variable'
)
display(km_200|raw_200)

In [ ]:
smooth_param=99
grad_raw=raw_data.drop('wavelength',axis=1).apply(first_derivative,axis=0,raw=True,wavelength=raw_data['wavelength'].values)
grad_raw['wavelength']=raw_data['wavelength']
grad_raw_smooth=raw_data.drop('wavelength',axis=1).apply(first_derivative_smooth,axis=0,raw=True,wavelength=raw_data['wavelength'].values,wl=smooth_param)
grad_raw_smooth['wavelength']=raw_data['wavelength']
grad_km=km_transformed.drop('wavelength',axis=1).apply(first_derivative,axis=0,raw=True,wavelength=km_transformed['wavelength'].values)
grad_km['wavelength']=km_transformed['wavelength']
grad_km_smooth=km_transformed.drop('wavelength',axis=1).apply(first_derivative_smooth,axis=0,raw=True,wavelength=km_transformed['wavelength'].values,wl=smooth_param)
grad_km_smooth['wavelength']=km_transformed['wavelength']

In [ ]:
grad_raw_200=alt.Chart(grad_raw.loc[:,['wavelength',100,1,0.1,0]].melt(id_vars=['wavelength'])).mark_line().encode(
    x=alt.X('wavelength'),
    y=alt.Y('value',axis=alt.Axis(title='d/dl % Reflectance')),
    color=alt.Color('variable',scale=alt.Scale(scheme='blues'))
)

grad_km_200=alt.Chart(grad_km.loc[:,['wavelength',100,1,0.1,0]].melt(id_vars=['wavelength'])).mark_line().encode(
    x=alt.X('wavelength'),
    y=alt.Y('value',axis=alt.Axis(title='d/dl K-M function')),
    color=alt.Color('variable',scale=alt.Scale(scheme='blues'))
)
grad_raw_s_200=alt.Chart(grad_raw_smooth.loc[:,['wavelength',100,1,0.1,0]].melt(id_vars=['wavelength'])).mark_line().encode(
    x=alt.X('wavelength'),
    y=alt.Y('value',axis=alt.Axis(title='d/dl % Reflectance')),
    color=alt.Color('variable',scale=alt.Scale(scheme='reds'))
)

grad_km_s_200=alt.Chart(grad_km_smooth.loc[:,['wavelength',100,1,0.1,0]].melt(id_vars=['wavelength'])).mark_line().encode(
    x=alt.X('wavelength'),
    y=alt.Y('value',axis=alt.Axis(title='d/dl K-M function')),
    color=alt.Color('variable',scale=alt.Scale(scheme='reds'))
)
display(((grad_km_s_200)|(grad_raw_s_200))
        & 
       (km_200|raw_200))

In [ ]:
def peaks(x,wavelength, smooth_param_peaks=smooth_param):
    p,_=sig.find_peaks(-x,width=smooth_param_peaks)
    if p.size >0:
        x0 = x[p].item()
        wavelength0 = wavelength[p].item()
    else:
        x0 = np.nan
        wavelength0 = np.nan
    return [x0,wavelength0]

grad_km_peaks=pd.DataFrame(columns=['peak','wavelength'],index=grad_km_smooth.drop('wavelength',axis=1).columns)
grad_km_peaks[['peak','wavelength']]=pd.Series(grad_km_smooth.drop('wavelength',axis=1).apply(peaks,axis=0,wavelength=grad_km_smooth['wavelength'],raw=True,result_type='expand')).tolist()
print(grad_km_peaks)

In [ ]:
transfer_function=alt.Chart(grad_km_peaks.reset_index()).mark_point().encode(
    x='index',
    y='peak'
)
transfer_regression=alt.Chart(grad_km_peaks[grad_km_peaks.index>=1].reset_index()).encode(
    x='index',
    y='peak'
).transform_regression(on='index',regression='peak',method='log').mark_line(color='black')
display(transfer_function+transfer_regression)